### Earth weather data since 1940 (max allowed) from Open-Meteo's API

For description of  locations considered and overall process, check notebook [earth_weather_data_group_1.ipynb][mylink]

[mylink]: earth_weather_data_group1.ipynb

In [1]:
#____ Library Import ____
import openmeteo_requests         # Open-Meteo weather API
import requests_cache             # chaching responses to reduce repetitive requests
import pandas as pd
from datetime import datetime, timezone
import time
from retry_requests import retry

In [2]:
#____ Caching & Client ____

cache_session = requests_cache.CachedSession(".cache", expire_after = -1)   # caching object; doesn't expire (-1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)     # in case the request fails
openmeteo = openmeteo_requests.Client(session = retry_session)              # Open-Meteo client

In [3]:
#____ Coords to Process ____

coordinates = [
    {"name": "Athens", "latitude": 37.9838, "longitude": 23.7275},       # Athens, Greece 
    {"name": "Serengeti", "latitude": -2.3260, "longitude": 34.8487},    # Serengeti National Park, Tanzania
    {"name": "Everest", "latitude": 27.9861, "longitude": 86.9226},      # Mount Everest Base Camp, Nepal
    {"name": "Sydney", "latitude": -33.8688, "longitude": 151.2093}      # Sydney, Australia
]

In [4]:
#____ Setup ____
url = "https://archive-api.open-meteo.com/v1/archive"       # base URL

In [5]:
now_utc = datetime.now(timezone.utc) 
end_date = now_utc.strftime("%Y-%m-%d")  

In [6]:
# params contains the query details 
base_params = {
    "start_date": "1940-01-01",
    "end_date": "2024-03-16",
    "hourly": ["temperature_2m", "relative_humidity_2m", "rain", "direct_radiation_instant"]
}

In [7]:
#____ Request ____
results = []  
hourly_dataframe = pd.DataFrame()

for coords in coordinates:
    params = base_params.copy()  # making a copy to avoid modifying the base parameters
    loc_name = coords["name"]
    params["latitude"] = coords["latitude"]
    params["longitude"] = coords["longitude"]


    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  

    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()}m ASL (*Above Sea Level)")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()}s")

    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_rain = hourly.Variables(2).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(3).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["latitude"] = params["latitude"]
    hourly_data["longitude"] = params["longitude"]
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["rain"] = hourly_rain
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant


    results.append({
        "coordinates": coords, 
        "weather_data": hourly_dataframe.to_dict("records") 
    })

    temp_df = pd.DataFrame(hourly_data) 

 
    hourly_dataframe = pd.concat([hourly_dataframe, temp_df], ignore_index=True) 
    hourly_dataframe.to_json(f"../data/weather_data_{loc_name}.json", orient="records")

    time.sleep(60)  

Coordinates 37.996482849121094°N 23.70967674255371°E
Elevation 92.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
zzZzZzZ
Coordinates -2.355008840560913°N 34.82014465332031°E
Elevation 1412.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
zzZzZzZ
Coordinates 28.01405906677246°N 86.95259857177734°E
Elevation 8395.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
zzZzZzZ
Coordinates -33.84885787963867°N 151.1955108642578°E
Elevation 86.0m ASL (*Above Sea Level)
Timezone None None
Timezone difference to GMT+0 0s
zzZzZzZ


In [8]:
hourly_dataframe

,date,latitude,longitude,temperature_2m,relative_humidity_2m,rain,direct_radiation_instant
0,1940-01-01 00:00:00+00:00,37.9838,23.7275,3.870000,84.342232,NaN,NaN
1,1940-01-01 01:00:00+00:00,37.9838,23.7275,3.470000,86.439995,NaN,NaN
2,1940-01-01 02:00:00+00:00,37.9838,23.7275,3.420000,86.434860,NaN,NaN
3,1940-01-01 03:00:00+00:00,37.9838,23.7275,4.170000,84.679268,NaN,NaN
4,1940-01-01 04:00:00+00:00,37.9838,23.7275,4.170000,84.377243,NaN,NaN
...,...,...,...,...,...,...,...
2952667,2024-03-16 19:00:00+00:00,-33.8688,151.2093,17.860001,89.791237,0.8,0.000000
2952668,2024-03-16 20:00:00+00:00,-33.8688,151.2093,18.110001,88.673149,0.3,0.000000
2952669,2024-03-16 21:00:00+00:00,-33.8688,151.2093,18.360001,87.850868,0.1,15.954878
2952670,2024-03-16 22:00:00+00:00,-33.8688,151.2093,18.610001,87.872688,0.1,0.000000
